Import dependencies

In [ ]:
%reload_ext autoreload
%autoreload 1
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import cobra

from yeast8model import Yeast8Model

# Construct models of cells of interest

In [ ]:
glc_exch_rate = 16.89
wt_ec = Yeast8Model("./models/ecYeastGEM_batch_8-6-0.xml")
wt_ec.model.reactions.get_by_id("r_1714").bounds = (-glc_exch_rate, 0)
wt_ec.model.reactions.get_by_id("r_1714_REV").bounds = (0, glc_exch_rate)

In [ ]:
from yeast8model import biomass_component_list_orig

glucose_bounds = (-4.75, 0) # gives a sensible growth rate for wt
wt_y8 = Yeast8Model("./models/yeast-GEM_8-6-0.xml", growth_id="r_2111", biomass_id="r_4041")
wt_y8.biomass_component_list = biomass_component_list_orig
wt_y8.model.reactions.r_1714.bounds = glucose_bounds
wt_y8.add_media_components(['r_1992'])

# Effect of flux constraints

In [ ]:
from yeast8model import get_ablation_ratio

Convenience functions

In [ ]:
def cf_vs_gr(ymodel, fractions):
    # get original flux sum
    sol = ymodel.optimize()
    orig_flux_sum = sol.fluxes.abs().sum()
    
    gr_list = []
    ratio_list = []
    for fraction in fractions:
        # impose flux constraint
        ub = fraction*orig_flux_sum
        ymodel.set_flux_constraint(upper_bound=ub)
        sol = ymodel.optimize()
        # ablate & ratio
        abl_res = ymodel.ablate()
        ratio = get_ablation_ratio(abl_res)
        # append lists
        gr_list.append(sol.fluxes['r_2111'])
        ratio_list.append(ratio)
        
    return gr_list, ratio_list

## Parameters

In [ ]:
# Step down, because 'phantom' values or copying issues (that I don't want to fix)
fractions = np.linspace(1, 0, num=51)

## Enzyme-constrained

Get sum of absolute values in 'vanilla' FBA.

Then, sweep constraint value, see if it affects ablation.

Note: re-initialise the model before (re-)running these.

In [ ]:
gr_list, ratio_list = cf_vs_gr(wt_ec, fractions)

In [ ]:
fig, ax = plt.subplots()
ax.plot(fractions, gr_list, 'o-')
ax.set_xlabel('Fraction of original sum of absolute values of fluxes')
ax.set_ylabel('Growth rate')
ax.set_xlim((0,1))
ax.set_ylim((0,0.5))

In [ ]:
fig, ax = plt.subplots()
ax.plot(fractions, ratio_list, 'o-')
ax.set_xlabel('Fraction of original sum of absolute values of fluxes')
ax.set_ylabel('Ratio')
ax.set_xlim((0,1))
ax.set_ylim((0,1.4))

In [ ]:
ratio_list

## Non-ec

In [ ]:
gr_list, ratio_list = cf_vs_gr(wt_y8, fractions)

In [ ]:
fig, ax = plt.subplots()
ax.plot(fractions, gr_list, 'o-')
ax.set_xlabel('Fraction of original sum of absolute values of fluxes')
ax.set_ylabel('Growth rate')
ax.set_xlim((0,1))
ax.set_ylim((0,0.5))

In [ ]:
fig, ax = plt.subplots()
ax.plot(fractions, ratio_list, 'o-')
ax.set_xlabel('Fraction of original sum of absolute values of fluxes')
ax.set_ylabel('Ratio')
ax.set_xlim((0,1))
ax.set_ylim((0,1.4))

# Unused

In [ ]:
upper_bound_list = []
flux_sum_list = []
ratio_list = []

for fraction in fractions:
    ub = fraction*orig_flux_sum
    wt_ec.set_flux_constraint(upper_bound=ub)
    sol = wt_ec.optimize()
    abl_res = wt_ec.ablate()
    ratio = get_ablation_ratio(abl_res)
    upper_bound_list.append(ub)
    flux_sum_list.append(sol.fluxes.abs().sum())
    ratio_list.append(ratio)